In [1]:
import pyterrier as pt
if not pt.started():
    pt.init()

from pyterrier_colbert.ranking import ColBERTFactory
pytcolbert = ColBERTFactory("http://www.dcs.gla.ac.uk/~craigm/ecir2021-tutorial/colbert_model_checkpoint.zip", "./msmarco_index", "msmarco", gpu=True)

msmarco_ds = pt.get_dataset("msmarco_passage")

# Get list 
qids = list(msmarco_ds.get_qrels("dev").loc[msmarco_ds.get_qrels("dev")['docno'].astype(int) < 100000]['qid'])

PyTerrier 0.9.2 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.
/home/stu1/s8/gfp4857/miniconda3/envs/project/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at bert-base-uncased were not used when initializing ColBERT: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing ColBERT from the checkpoint of a model trained on another task or with another architecture (e.

[May 06, 15:58:21] #> Loading model checkpoint.
[May 06, 15:58:21] #> Loading checkpoint http://www.dcs.gla.ac.uk/~craigm/ecir2021-tutorial/colbert_model_checkpoint.zip


/home/stu1/s8/gfp4857/miniconda3/envs/project/lib/python3.11/site-packages/torch/hub.py:665: UserWarning: Falling back to the old format < 1.6. This support will be deprecated in favor of default zipfile format introduced in 1.6. Please redo torch.save() to save it in the new zipfile format.
  warnings.warn('Falling back to the old format < 1.6. This support will be '


[May 06, 15:58:50] #> checkpoint['epoch'] = 0
[May 06, 15:58:50] #> checkpoint['batch'] = 44500


In [18]:
ids = pytcolbert.unique_token_ids

bert_ids = []
for id, tok in zip(ids, pytcolbert.token_ids_to_strings(ids)):
    if tok.startswith('[') and tok.endswith(']'):
        print(tok, id)
        bert_ids += [id]
pytcolbert.token_ids_to_strings(bert_ids)

[CLS] 101
[SEP] 102
[D] 2
[UNK] 100


['[CLS]', '[SEP]', '[D]', '[UNK]']

In [15]:
from ir_measures import RR

dense_e2e = pytcolbert.end_to_end()
dense_e2e_bert_pruned = pytcolbert.end_to_end(bert_ids)

pt.Experiment(
    [dense_e2e, dense_e2e_bert_pruned],
    msmarco_ds.get_topics("dev").loc[msmarco_ds.get_topics("dev")['qid'].str.contains('|'.join(qids), na=False)],
    msmarco_ds.get_qrels("dev"),
    eval_metrics=["map", RR@10],
    baseline=0,
    names=["No Pruning", "BERT Tokens Pruned"]
)

DEBUG>>> token_ids_to_prune=[101, 102, 2, 100]
DEBUG>>> vectors_ids_to_prune=array([      0,       1,      55, ..., 7409270, 7409271, 7409336])
DEBUG>>> pre-remove: 7409337
DEBUG>>> post-remove: 7108562


,name,map,RR@10,map +,map -,map p-value,RR@10 +,RR@10 -,RR@10 p-value
0,No Pruning,0.481221,0.507304,NaN,NaN,NaN,NaN,NaN,NaN
1,BERT Tokens Pruned,0.480934,0.507612,22.0,13.0,0.943562,10.0,10.0,0.944659


In [20]:
from ir_measures import RR

bert_ids = [101, 102]

dense_e2e = pytcolbert.end_to_end()
dense_e2e_bert_pruned = pytcolbert.end_to_end(bert_ids)

pt.Experiment(
    [dense_e2e, dense_e2e_bert_pruned],
    msmarco_ds.get_topics("dev").loc[msmarco_ds.get_topics("dev")['qid'].str.contains('|'.join(qids), na=False)],
    msmarco_ds.get_qrels("dev"),
    eval_metrics=["map", RR@10],
    baseline=0,
    names=["No Pruning", "BERT Tokens Pruned"]
)

DEBUG>>> token_ids_to_prune=[101, 102]
DEBUG>>> vectors_ids_to_prune=array([      0,      55,      56, ..., 7409269, 7409270, 7409336])
DEBUG>>> pre-remove: 7409337
DEBUG>>> post-remove: 7209337


,name,map,RR@10,map +,map -,map p-value,RR@10 +,RR@10 -,RR@10 p-value
0,No Pruning,0.481221,0.507304,NaN,NaN,NaN,NaN,NaN,NaN
1,BERT Tokens Pruned,0.483088,0.509125,13.0,11.0,0.380305,6.0,7.0,0.418388


In [21]:
from ir_measures import RR

dense_e2e = pytcolbert.end_to_end()

token_ids_p25 = pytcolbert.top_p_tokens(0.25)
dense_e2e_p25 = pytcolbert.end_to_end(token_ids_p25)

pt.Experiment(
    [dense_e2e, dense_e2e_p25],
    msmarco_ds.get_topics("dev").loc[msmarco_ds.get_topics("dev")['qid'].str.contains('|'.join(qids), na=False)],
    msmarco_ds.get_qrels("dev"),
    eval_metrics=["map", RR@10],
    baseline=0,
    names=["No Pruning", "25% Pruning"]
)

DEBUG>>> token_ids_to_prune={101, 102, 1996, 1997, 1998, 1037, 2000, 1999, 1010, 1012}
DEBUG>>> vectors_ids_to_prune=array([      0,       2,       4, ..., 7409331, 7409335, 7409336])
DEBUG>>> pre-remove: 7409337
DEBUG>>> post-remove: 5546301


,name,map,RR@10,map +,map -,map p-value,RR@10 +,RR@10 -,RR@10 p-value
0,No Pruning,0.481221,0.507304,NaN,NaN,NaN,NaN,NaN,NaN
1,25% Pruning,0.487362,0.515169,26.0,33.0,0.204736,17.0,23.0,0.169815
